# Project 3 

#### Jason Kim 
#### Braulio Lopez

### Part 1 

#### Importing the libraries

In [1]:
pip install tensorflow_datasets --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from pathlib import Path
import keras
from keras.datasets import mnist
import random
import shutil
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

/tmp/ipykernel_7497/952184717.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-11 05:15:49.691033: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 05:15:49.750011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 05:15:49.750071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin c

#### Loading Dataset 

In [3]:
try:
    shutil.rmtree("data_all_modified-cnn-split/train")
    shutil.rmtree("data_all_modified-cnn-split/test")
except:
    pass

In [4]:
Path("data_all_modified-cnn-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-cnn-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data_all_modified-cnn-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-cnn-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [5]:
all_damage_file_paths = os.listdir('data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data_all_modified/no_damage')

In [6]:
train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [7]:
# ensure to copy the images to the directories
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-cnn-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-cnn-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-cnn-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-cnn-split/test/no_damage', p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("data_all_modified-cnn-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data_all_modified-cnn-split/train/no_damage")))
print("Files in test/damage: ", len(os.listdir("data_all_modified-cnn-split/test/damage")))
print("Files in test/no_damage: ", len(os.listdir("data_all_modified-cnn-split/test/no_damage")))

Files in train/damage:  11336
Files in train/no_damage:  5721
Files in test/damage:  2834
Files in test/no_damage:  1431


#### Data Processing

In [8]:
from PIL import Image
import os

# Path to the directory containing your images
dataset_dir = 'data_all_modified/no_damage'

image_dimensions = {}

# Loop through each image file in the directory
for filename in os.listdir(dataset_dir):
    if filename.endswith('.jpeg') or filename.endswith('.png'):  # Adjust file extensions as needed
        # Open the image using PIL
        img = Image.open(os.path.join(dataset_dir, filename))
        # Get the dimensions of the image
        width, height = img.size
        # Add dimensions to the dictionary
        if (width, height) in image_dimensions:
            image_dimensions[(width, height)] += 1
        else:
            image_dimensions[(width, height)] = 1

# Print summary of image dimensions
for dimensions, count in image_dimensions.items():
    print(f"{count} images with dimensions {dimensions[0]}x{dimensions[1]}")

7152 images with dimensions 128x128


In [9]:
train_data_dir = 'data_all_modified-cnn-split/train'
batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [10]:
test_data_dir = 'data_all_modified-cnn-split/test/'

batch_size = 2

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4265 files belonging to 2 classes.


### Part 2

#### ANN

In [24]:
from tensorflow.keras import models, layers

model_ann = models.Sequential()

# Adjust the input_shape to match the actual image size
model_ann.add(layers.Flatten(input_shape=(128, 128, 3)))  # This line is changed

model_ann.add(layers.Dense(512, activation='relu'))
model_ann.add(layers.Dense(256, activation='relu'))
model_ann.add(layers.Dense(1, activation='softmax'))  # For binary classification

model_ann.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model_ann.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 49152)             0         
                                                                 
 dense_8 (Dense)             (None, 512)               25166336  
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 dense_10 (Dense)            (None, 1)                 257       
                                                                 
Total params: 25297921 (96.50 MB)
Trainable params: 25297921 (96.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
history_ann = model_ann.fit(
    train_rescale_ds,
    epochs=20,
    validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 57s 131ms/step - loss: 1.2020 - accuracy: 0.6326 - val_loss: 0.6359 - val_accuracy: 0.6681
Epoch 2/20
427/427 [==============================] - 60s 141ms/step - loss: 0.6408 - accuracy: 0.6637 - val_loss: 0.6359 - val_accuracy: 0.6681
Epoch 3/20
427/427 [==============================] - 62s 145ms/step - loss: 0.6416 - accuracy: 0.6633 - val_loss: 0.6358 - val_accuracy: 0.6681
Epoch 4/20
427/427 [==============================] - 62s 144ms/step - loss: 0.6395 - accuracy: 0.6633 - val_loss: 0.6359 - val_accuracy: 0.6681
Epoch 5/20
427/427 [==============================] - 59s 138ms/step - loss: 0.6388 - accuracy: 0.6637 - val_loss: 0.6361 - val_accuracy: 0.6681
Epoch 6/20
427/427 [==============================] - 67s 156ms/step - loss: 0.6388 - accuracy: 0.6637 - val_loss: 0.6360 - val_accuracy: 0.6681
Epoch 7/20
427/427 [==============================] - 65s 153ms/step - loss: 0.6389 - accuracy: 0.6637 - val_loss: 0.6359 - val_ac

In [13]:
test_loss, test_accuracy = model_ann.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.6381529569625854
Test Accuracy: 0.6644783020019531


In [15]:
from pathlib import Path
from tensorflow.keras.models import load_model

# Define the path where you want to save the model
model_path = Path('./models/ANN_Model.h5')

# Check if the file already exists
if model_path.exists():
    # If the file exists, remove it
    model_path.unlink()

# Save the model
model_ann.save(model_path)

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### LeNet-5

In [16]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(3, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d (Average  (None, 63, 63, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_1 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 14400)             0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [17]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 25s 56ms/step - loss: 0.6100 - accuracy: 0.6958 - val_loss: 0.5545 - val_accuracy: 0.7737
Epoch 2/20
427/427 [==============================] - 24s 56ms/step - loss: 0.4931 - accuracy: 0.7816 - val_loss: 0.4691 - val_accuracy: 0.7663
Epoch 3/20
427/427 [==============================] - 23s 54ms/step - loss: 0.4298 - accuracy: 0.8253 - val_loss: 0.3785 - val_accuracy: 0.8543
Epoch 4/20
427/427 [==============================] - 23s 54ms/step - loss: 0.3991 - accuracy: 0.8399 - val_loss: 0.3625 - val_accuracy: 0.8581
Epoch 5/20
427/427 [==============================] - 22s 51ms/step - loss: 0.3796 - accuracy: 0.8489 - val_loss: 0.5462 - val_accuracy: 0.7529
Epoch 6/20
427/427 [==============================] - 24s 56ms/step - loss: 0.3628 - accuracy: 0.8578 - val_loss: 0.4857 - val_accuracy: 0.7886
Epoch 7/20
427/427 [==============================] - 24s 56ms/step - loss: 0.3442 - accuracy: 0.8677 - val_loss: 0.4702 - val_accuracy:

In [18]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.5797984004020691
Test Accuracy: 0.7995310425758362


In [19]:
from pathlib import Path
from tensorflow.keras.models import load_model

# Define the path where you want to save the model
model_path = Path('./models/Lenet5_cnn.h5')

# Check if the file already exists
if model_path.exists():
    # If the file exists, remove it
    model_path.unlink()

# Save the model
model_lenet5.save(model_path)

#### Alternate-Lenet-5 CNN architecture

In [11]:
from keras.layers import LeakyReLU
from keras.regularizers import l2
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

model = models.Sequential()

model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (128,128,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation = 'relu', kernel_regularizer = l2(1e-4)))
model.add(layers.Dense(1, activation = 'sigmoid'))

#compile the model with RMSprob with learning rate
from keras import optimizers
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(learning_rate=1e-4), metrics = ['acc'])

#process the jpeg image
#create an image generator
from keras.preprocessing.image import ImageDataGenerator

#train using data augmentation and dropout
train_datagen = ImageDataGenerator(
                    rescale = 1./255,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255) #validation data should not be augmented

train_generator = train_datagen.flow_from_directory(
                    'data_all_modified-cnn-split/train', 
                    target_size = (128,128),
                    batch_size = 32,
                    class_mode = 'binary')
validation_generator = test_datagen.flow_from_directory(
                    'data_all_modified-cnn-split/test', 
                    target_size = (128,128),
                    batch_size = 32,
                    class_mode = 'binary')

model.summary()

Found 17057 images belonging to 2 classes.
Found 4265 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                     

In [12]:
history = model.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 104s 239ms/step - loss: 0.7165 - acc: 0.6721 - val_loss: 0.6858 - val_acc: 0.6714
Epoch 2/20
427/427 [==============================] - 97s 227ms/step - loss: 0.6099 - acc: 0.7516 - val_loss: 0.7441 - val_acc: 0.6807
Epoch 3/20
427/427 [==============================] - 97s 228ms/step - loss: 0.5376 - acc: 0.8004 - val_loss: 0.7965 - val_acc: 0.6889
Epoch 4/20
427/427 [==============================] - 94s 221ms/step - loss: 0.4741 - acc: 0.8371 - val_loss: 0.6427 - val_acc: 0.7347
Epoch 5/20
427/427 [==============================] - 98s 230ms/step - loss: 0.4058 - acc: 0.8677 - val_loss: 0.7841 - val_acc: 0.7256
Epoch 6/20
427/427 [==============================] - 95s 223ms/step - loss: 0.3375 - acc: 0.9000 - val_loss: 0.5676 - val_acc: 0.8015
Epoch 7/20
427/427 [==============================] - 96s 224ms/step - loss: 0.2960 - acc: 0.9123 - val_loss: 0.7068 - val_acc: 0.7748
Epoch 8/20
427/427 [==============================] - 

In [13]:
test_loss, test_accuracy = model.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.18474186956882477
Test Accuracy: 0.9502930641174316


In [15]:
from pathlib import Path
from tensorflow.keras.models import load_model

# Define the path where you want to save the model
model_path = Path('./models/Alternate_Lenet5.h5')

# Check if the file already exists
if model_path.exists():
    # If the file exists, remove it
    model_path.unlink()

# Save the model
model.save(model_path)

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
model.save('./models/Alternate_Lenet5.keras')

In [20]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

model = load_model('./models/Alternate_Lenet5.keras')

# Read the image file from the request
img_file = './data_all_modified-cnn-split/test/damage/-95.588835_29.769012.jpeg'

# Open the image using PIL
img = Image.open(img_file)
        
# Resize and preprocess the image
img = img.resize((128, 128))
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Perform inference using the pre-trained model
prediction = model.predict(img_array)

# Extract the probability of being damaged
probability_damaged = prediction[0][0]

# Determine the class based on the threshold
threshold = 0.5
if probability_damaged >= threshold:
    prediction_label = 'No Damage'
else:
    prediction_label = 'Damage'

# Convert the prediction result into a JSON format
result = {
    'probability_no_damage': float(probability_damaged),
    'prediction': prediction_label
}

result


1/1 [==============================] - 0s 100ms/step


{'probability_no_damage': 9.33332703425549e-05, 'prediction': 'Damage'}

In [16]:
train_ds.class_names

['damage', 'no_damage']

In [22]:
prediction

array([[9.333327e-05]], dtype=float32)

In [24]:
model.predict(test_ds).tolist()

134/134 [==============================] - 7s 48ms/step


[[0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],


In [25]:
import requests
import json

# Define the URL of your Flask server endpoint
url = 'http://localhost:5000/predict_and_evaluate_dataset'

# Define the path to the dataset directory containing 'damage' and 'no_damage' subdirectories
dataset_dir = '/home/ubuntu/nb-data/Projects/Project3/src/data_all_modified-cnn-split/test'

# Send a POST request to the Flask server
response = requests.post(url, params={'dataset_dir': dataset_dir})

# Check the response status code
if response.status_code == 200:
    # Print the response content (JSON data)
    print(response.json())
else:
    # Print the error message if the request was unsuccessful
    print("Error:", response.text)


ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /predict_and_evaluate_dataset?dataset_dir=%2Fhome%2Fubuntu%2Fnb-data%2FProjects%2FProject3%2Fsrc%2Fdata_all_modified-cnn-split%2Ftest (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f876eb9ebd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [26]:
requests.get('http://localhost:5000/model_summary')

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /model_summary (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f876e0a37d0>: Failed to establish a new connection: [Errno 111] Connection refused'))